### Example pipelines using data from Andrew Ng's Machine Learning course on Coursera

The first example uses GridSearchCV to find the best hyperparameters for the NN.
The second example produces a learning curve to help diagnose bias/variance problems.

In [1]:
from sklearn.model_selection import train_test_split
import scipy.io
import numpy as np

from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

data = scipy.io.loadmat('../ngml/ex4data1.mat')
X = np.array(data.get('X'))
y = np.ravel(np.array(data.get('y')))

# randomly split the data into training, cross validation and test sets
random_state = 42

X_train, X_blind, y_train, y_blind = train_test_split(
    X, y, test_size=0.4, random_state=random_state)

X_test, X_cv, y_test, y_cv = train_test_split(
    X_blind, y_blind, test_size=0.5, random_state=random_state)

params = { 'hidden_layer_sizes' : [[249, 249], [200]],
    'activation' : ['relu'], 'solver' : ['adam'],
    'alpha': [.1],
    'random_state' : [random_state],
    'learning_rate' : ['constant'],
    'learning_rate_init' : [0.01],
    'max_iter' : [1000], 'shuffle' : [True],
    'n_iter_no_change' : [50]}

# model = make_pipeline(MLPClassifier(**params))
nn = MLPClassifier()
clf = GridSearchCV(nn, params, verbose=2, n_jobs=11)
model = make_pipeline(StandardScaler(), clf)
model.fit(X_train, y_train)


P_train = model.predict(X_train)
P_test = model.predict(X_test)
P_cv = model.predict(X_cv)

err_train = mean_squared_error(y_train, P_train)
err_train = mean_squared_error(y_test, P_test)
err_cv = mean_squared_error(y_cv, P_cv)

score_train = model.score(X_train, y_train)
score_test = model.score(X_test, y_test)
score_cv = model.score(X_cv, y_cv)

print('MSE train:', err_train)
print('MSE test:', err_train)
print('MSE cv:', err_cv)

print('Score train:', score_train)
print('Score test:', score_test)
print('Score cv:', score_cv)

print('Best parameters:', clf.best_params_)
print('Best score:', clf.best_score_)
print('Best estimator:', clf.best_estimator_)
print('CV results:', clf.cv_results_)
print('CV results keys:', clf.cv_results_.keys())


Fitting 5 folds for each of 2 candidates, totalling 10 fits
MSE train: 1.208
MSE test: 1.208
MSE cv: 0.892
Score train: 1.0
Score test: 0.925
Score cv: 0.932
Best parameters: {'activation': 'relu', 'alpha': 0.1, 'hidden_layer_sizes': [249, 249], 'learning_rate': 'constant', 'learning_rate_init': 0.01, 'max_iter': 1000, 'n_iter_no_change': 50, 'random_state': 42, 'shuffle': True, 'solver': 'adam'}
Best score: 0.929
Best estimator: MLPClassifier(alpha=0.1, hidden_layer_sizes=[249, 249], learning_rate_init=0.01,
              max_iter=1000, n_iter_no_change=50, random_state=42)
CV results: {'mean_fit_time': array([25.89307861, 14.9640873 ]), 'std_fit_time': array([3.45381704, 2.99169843]), 'mean_score_time': array([0.00620809, 0.01196127]), 'std_score_time': array([0.00536412, 0.00709563]), 'param_activation': masked_array(data=['relu', 'relu'],
             mask=[False, False],
       fill_value='?',
            dtype=object), 'param_alpha': masked_array(data=[0.1, 0.1],
             mas

Example of creating a learning curve

In [ ]:
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import scipy.io
import numpy as np
import pandas as pd
import math

from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_squared_error
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler

data = scipy.io.loadmat('../ngml/ex4data1.mat')
X = np.array(data.get('X'))
y = np.ravel(np.array(data.get('y')))

# randomly split the data into training, cross validation and test sets
random_state = 42

X_train, X_blind, y_train, y_blind = train_test_split(
    X, y, test_size=0.4, random_state=random_state)

X_test, X_cv, y_test, y_cv = train_test_split(
    X_blind, y_blind, test_size=0.5, random_state=random_state)

params = { 'hidden_layer_sizes' : [249],
    'activation' : 'relu', 'solver' : 'adam',
    'random_state' : random_state, 'alpha' : 0.1,
    'learning_rate' : 'constant',
    'learning_rate_init' : 0.01,
    'max_iter' : 1000, 'shuffle' : True,
    'n_iter_no_change' : 50 }

model = make_pipeline(MLPClassifier(**params))

train_results = []
for i in range(5, X_train.shape[0], math.floor(X_train.shape[0] / 4)):
    model.fit(X_train[0:i, :], y_train[0:i])

    P_train = model.predict(X_train)
    P_test = model.predict(X_test)
    P = model.predict(X)

    err_train = mean_squared_error(y_train, P_train)
    err_test = mean_squared_error(y_test, P_test)
    score_all = model.score(X, y)
    score_test = model.score(X_test, y_test)

    print(f'n={i} train: {err_train:0.4} test: {err_test:0.4} score: {score_test:0.4}')
    train_results.append([i, err_train, err_test, score_all, score_test])

if i < X_train.shape[0]:
    i = X_train.shape[0]
    model.fit(X_train, y_train)

    P_train = model.predict(X_train)
    P_test = model.predict(X_test)
    P = model.predict(X)

    err_train = mean_squared_error(y_train, P_train)
    err_test = mean_squared_error(y_test, P_test)
    score_all = model.score(X, y)
    score_test = model.score(X_test, y_test)

    print(f'n={i} train: {err_train:0.4} test: {err_test:0.4} score: {score_test:0.4}')
    train_results.append([i, err_train, err_test, score_all, score_test])

train_results = np.array(train_results)
score_cv = model.score(X_cv, y_cv)

trdf = pd.DataFrame(train_results, columns=['n', 'train', 'test', 'score_all', 'score_test'])
trdf.plot(x='n', y=['train', 'test', 'score_all', 'score_test'])
plt.title(f'Error vs Amt of training (n samples)\n(CV accuracy: {score_cv:0.2})')
plt.show()